In [5]:
# 1. 필요 라이브러리 설치
# 이 스크립트를 실행하기 전에 터미널에서 아래 명령어를 실행하여 필요한 라이브러리를 설치해주세요.
# pip install -qU langchain-upstage langchain-core langchain faiss-cpu pypdf python-dotenv

import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA

def main_rag_pipeline():
    """
    콘텐츠 분쟁 해결 RAG 시스템의 전체 파이프라인을 실행하는 메인 함수입니다.
    """
    try:
        # --- 환경 설정 ---
        load_dotenv()
        print("환경 변수 로드 완료")

        # --- 0단계: 문서 로드 ---
        # ⛔️ 오류 수정: __file__ 대신 직접 상대 경로를 지정합니다.
        # 이 코드를 실행하는 위치가 MyLangChainProject 폴더 최상단이라고 가정합니다.
        pdf_path = 'src/mylangchain_app/data/콘텐츠분쟁해결_사례.pdf'
        loader = PyPDFLoader(pdf_path)
        docs = loader.load()
        print(f"{len(docs)}개의 문서 로드 완료")

        # --- 1단계: 문서 분할 ---
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1500,
            chunk_overlap=300,
            separators=[
                "\\n【사건개요】", "\\n【쟁점사항】", "\\n【처리경위】", "\\n【처리결과】",
                "\\n■", "\\n\\n", "\\n", ".", " ", ""
            ]
        )
        splits = text_splitter.split_documents(docs)
        print(f"{len(splits)}개의 분할된 문서 생성 완료")

        # --- 2단계: 임베딩 모델 설정 ---
        embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
        print("임베딩 모델 설정 완료")

        # --- 3단계: 벡터 저장소 및 검색기 설정 ---
        vectorstore = FAISS.from_documents(splits, embeddings)
        # ⛔️ SyntaxError 수정: 'as_ retriever'의 공백을 제거하여 'as_retriever'로 수정합니다.
        retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})
        print("벡터 저장소 및 검색기 설정 완료")

        # --- 4단계: LLM 설정 ---
        llm = ChatUpstage(model_name="solar-1-mini-chat", temperature=0.2)
        print("LLM 설정 완료")

        # --- 5단계: 법률 자문 프롬프트 작성 ---
        prompt_template = """
        당신은 콘텐츠 분야 전문 법률 자문가입니다. 
        아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.
        관련 분쟁사례: {context}
        상담 내용: {question}
        답변 가이드라인:
        1. 제시된 사례들을 근거로 답변하세요
        2. 관련 법령이나 조항이 있다면 명시하세요
        3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
        4. 실무적 해결방안을 단계별로 제시하세요
        5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요
        전문 법률 조언:"""
        prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_template)
        print("프롬프트 템플릿 생성 완료")

        # --- 6단계: QA 체인 생성 ---
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm, 
            chain_type="stuff",
            retriever=retriever,
            chain_type_kwargs={"prompt": prompt},
            return_source_documents=True
        )
        print("QA 체인 생성 완료\\n")

        # --- 7단계: 테스트 질문 작성 및 실행 ---
        print("--- 법률 자문 테스트 시작 ---")
        test_questions = [
            "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
            "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
            "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
            "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
            "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
        ]
        for question in test_questions:
            print(f"질문: {question}")
            result = qa_chain.invoke(question)
            print(f"답변: {result['result']}")
            print("--------------------------------------------------\\n")
        print("--- 법률 자문 테스트 종료 ---\\n")

        # --- 8단계: 분쟁 유형 분류 함수 테스트 ---
        def classify_dispute_type(query):
            game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
            elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
            web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]
            query_lower = query.lower()
            if any(keyword in query_lower for keyword in game_keywords):
                return "게임"
            elif any(keyword in query_lower for keyword in elearning_keywords):
                return "이러닝"
            elif any(keyword in query_lower for keyword in web_keywords):
                return "웹콘텐츠"
            else:
                return "기타"
        
        print("--- 분쟁 유형 분류 테스트 시작 ---")
        for question in test_questions:
            dispute_type = classify_dispute_type(question)
            print(f"질문: {question}")
            print(f"분류된 분쟁 유형: {dispute_type}\\n")
        print("--- 분쟁 유형 분류 테스트 종료 ---")
            
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

if __name__ == '__main__':
    # 스크립트가 직접 실행될 때만 메인 함수를 호출합니다.
    main_rag_pipeline()



환경 변수 로드 완료
오류가 발생했습니다: File path src/mylangchain_app/data/콘텐츠분쟁해결_사례.pdf is not a valid file or url
